# Evaluation

## Open dataset

In [30]:
from pathlib import Path
from constants import *

# Open source
with open(Path(DATASET_DIR, 'wmt14_en_test.src'), 'r') as f:
    source_dataset = []
    for sentence in f:
        source_dataset.append([int(x) for x in sentence.split(' ')[:-1]])

# Open target
with open(Path(DATASET_DIR, 'wmt14_fr_test.trg'), 'r') as f:
    target_dataset = []
    for sentence in f:
        target_dataset.append([int(x) for x in sentence.split(' ')[:-1]])


## Create the model

In [28]:
from models import Transformer

source_vocab_size = 20
target_vocab_size = 20
num_layers = 2
seq_length = 10
model = Transformer(embed_dim=512, src_vocab_size=source_vocab_size, 
                trg_vocab_size=target_vocab_size, seq_length=seq_length,
                num_layers_enc=num_layers, num_layers_dec=num_layers,
                hidden_size=2048, n_head=8, enc_dropout=0.2, dec_dropout=0.2)

## Make a prediction

In [29]:
import torch
from utils import create_mask

In [4]:
source = source_dataset[0]
target = target_dataset[0]

In [7]:
# Prediction 
source = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
max_seq_len = 512
source_mask = torch.ones(1, 1, source.size(1))
target = torch.tensor([[0]])
target_mask = create_mask(target.size(1)).type_as(source.data)
# Prediction = 
print(model.generate_greedy(source, target, source_mask, target_mask, max_seq_len))

/data4/aamayuelasfernandez/CS291K-project/layers.py:305: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.fully_connected_out(x))


tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


## Evaluation

In [9]:
# Open target
from pathlib import Path
from constants import *

with open(Path(DATASET_DIR, 'wmt14_fr_test.trg'), 'r') as f:
    target_dataset = []
    for sentence in f:
        target_dataset.append([int(x) for x in sentence.split(' ')[:-1]])

In [10]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(
        "gpt2",
        unk_token="<|unk|>",
        bos_token="<|bos|>",
        eos_token="<|eos|>", 
        )

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [18]:
txt_targets = []
for sentence in target_dataset:
    decoded_sentence = tokenizer.decode(sentence)
    txt_target = [token for token in decoded_sentence.split(' ') if (token != '<|bos|>' and token != '<|eos|>')]
    txt_targets.append(txt_target)


In [19]:
PREDICTIONS_DIR = "./predictions/"

with open(PREDICTIONS_DIR + "wmt14_en_fr.txt", 'r') as f:
    txt_predictions = f.readlines()

In [48]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = []
for i in range(len(txt_predictions)):
    score = sentence_bleu([txt_targets[i]], txt_predictions[i].split(" "), )
    bleu_scores.append(score)

print("BLEU score: ", sum(bleu_scores)/len(bleu_scores))

BLEU score:  0.9147635738284462
